**Task 2: BERT Fine-Tuning Strategies for Resume NER**

**Introduction**

Automated resume screening systems require accurate extraction of structured information from unstructured resume text. Named Entity Recognition (NER) using transformer-based models such as BERT has shown strong performance in information extraction tasks; however, full fine-tuning of these large models is computationally expensive and may lead to hallucinated entity extraction. Hallucination in resume parsing can negatively impact recruitment decisions and system reliability. This project explores parameter-efficient fine-tuning using Low-Rank Adaptation (LoRA) to improve resume NER performance while reducing computational cost and hallucination. The proposed approach aims to deliver a scalable and deployment-ready resume information extraction component.

**Objectives**

•	To develop a BERT-based NER system for extracting skills, education,
•	experience, and certifications from resumes.

•	To evaluate multiple LoRA fine-tuning configurations under identical experimental settings.

•	To compare LoRA-based models with a zero-shot BERT baseline.

•	To analyze extraction accuracy, computational efficiency, and hallucination behavior.

•	To identify an optimal trade-off between performance and deployment feasibility.


**Research Questions**

•	How does LoRA-based fine-tuning impact resume NER accuracy compared to zero-shot BERT? .

•	What is the effect of different LoRA ranks on performance and efficiency?

•	Does parameter-efficient fine-tuning reduce hallucinated entity extraction?

•	How well do LoRA-adapted models generalize to unseen resume data?


**Contribution to the Project**

•	Provides an empirical study of hallucination in resume-specific NER tasks.

•	Demonstrates the effectiveness of LoRA for efficient and accurate resume entity extraction.

•	Delivers a lightweight, deployable NER component for real-world recruitment systems.

•	Establishes a reproducible framework for future resume parsing research.


In [ ]:
#Libraries SETUP
!pip install -q transformers datasets peft accelerate nltk seqeval evaluate

import pandas as pd
import numpy as np
import torch
import random
import nltk
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("punkt_tab")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)


#LOAD DATASET
df = pd.read_csv("/content/AI_Resume_Screening.csv", encoding="latin-1")
print(df.columns.tolist())


#BUILD SYNTHETIC RESUME TEXT
def build_resume_text(row):
    return (
        f"Skills: {row['Skills']}. "
        f"Education: {row['Education']}. "
        f"Certifications: {row['Certifications']}. "
        f"Job Role: {row['Job Role']}."
    )


#BIO LABEL SCHEMA
label_list = [
    "O",
    "B-SKILL","I-SKILL",
    "B-EDU","I-EDU",
    "B-EXP","I-EXP",
    "B-CERT","I-CERT"
]

label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}


#BUILD NER SAMPLES
def build_ner_example(row):
    text = build_resume_text(row)
    tokens = word_tokenize(text)

    labels = []
    for tok in tokens:
        if tok in str(row["Skills"]):
            labels.append("B-SKILL")
        elif tok in str(row["Education"]):
            labels.append("B-EDU")
        elif tok in str(row["Certifications"]):
            labels.append("B-CERT")
        elif tok in str(row["Job Role"]):
            labels.append("B-EXP")
        else:
            labels.append("O")

    return {"tokens": tokens, "ner_tags": labels}


ner_data = [build_ner_example(row) for _, row in df.head(400).iterrows()]



#TRAIN / VAL / TEST SPLIT
from datasets import Dataset

dataset = Dataset.from_list(ner_data)
dataset = dataset.train_test_split(test_size=0.3, seed=42)
tmp = dataset["test"].train_test_split(test_size=0.5, seed=42)

dataset = {
    "train": dataset["train"],
    "validation": tmp["train"],
    "test": tmp["test"]
}



#TOKENIZATION + ALIGNMENT

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_align(examples):
    tokenized = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=512
    )

    labels = []
    for i, lab in enumerate(examples["ner_tags"]):
        word_ids = tokenized.word_ids(batch_index=i)
        prev = None
        label_ids = []

        for w in word_ids:
            if w is None:
                label_ids.append(-100)
            elif w != prev:
                label_ids.append(label2id[lab[w]])
            else:
                label_ids.append(
                    label2id[lab[w].replace("B-","I-")]
                    if lab[w] != "O" else label2id["O"]
                )
            prev = w

        labels.append(label_ids)

    tokenized["labels"] = labels
    return tokenized


tokenized_dataset = {
    k: v.map(tokenize_align, batched=True, remove_columns=v.column_names)
    for k,v in dataset.items()
}



#METRICS

import evaluate
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    preds, labels = p
    preds = preds.argmax(axis=-1)

    true_preds, true_labels = [], []
    for pred, lab in zip(preds, labels):
        cp, cl = [], []
        for p_i, l_i in zip(pred, lab):
            if l_i != -100:
                cp.append(id2label[p_i])
                cl.append(id2label[l_i])
        true_preds.append(cp)
        true_labels.append(cl)

    results = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"]
    }



#FUNCTION TO TRAIN LoRA MODEL

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

def train_lora_model(rank, alpha):
    model = AutoModelForTokenClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=len(label_list),
        id2label=id2label,
        label2id=label2id
    )

    lora = LoraConfig(
        r=rank,
        lora_alpha=alpha,
        target_modules=["query","value"],
        task_type="TOKEN_CLS"
    )

    model = get_peft_model(model, lora)

    args = TrainingArguments(
        output_dir=f"./lora_r{rank}",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        num_train_epochs=5,
        fp16=True,
        logging_steps=50
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    return trainer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['Resume_ID', 'Name', 'Skills', 'Experience (Years)', 'Education', 'Certifications', 'Job Role', 'Recruiter Decision', 'Salary Expectation ($)', 'Projects Count', 'AI Score (0-100)']


Map:   0%|          | 0/280 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]